In [ ]:
%matplotlib inline
import cv2
import numpy as np
import scipy
from scipy.ndimage import gaussian_laplace
from scipy.ndimage import rank_filter
from skimage.transform import resize
from matplotlib import pyplot as plt
import time

In [ ]:
def show_all_circles(image, cx, cy, rad, color='r'):
    """
    image: numpy array, representing the grayscsale image
    cx, cy: numpy arrays or lists, centers of the detected blobs
    rad: numpy array or list, radius of the detected blobs
    """
    import matplotlib.pyplot as plt
    from matplotlib.patches import Circle

    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.imshow(image, cmap='gray')
    for x, y, r in zip(cx, cy, rad):
        circ = Circle((x, y), r, color=color, fill=False)
        ax.add_patch(circ)

    plt.title('%i circles' % len(cx))
    plt.show()

In [ ]:
read_image = cv2.imread('assignment2_images/butterfly.jpg', 0)
# normalize to 0 to 1
read_image = read_image / 255
plt.imshow(read_image, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

In [ ]:
image_height = (read_image.shape)[0]
image_width = (read_image.shape)[1]

In [ ]:
# Change filter size
initial_sigma = 0.9
current_sigma = 0.9
scale = 1.3
num_level = 12
compare_size = 25
compare_height = 3

In [ ]:
start_time = time.time()
scale_space = np.empty((image_height, image_width, num_level)) # [h,w] - dimensions of image, n - number of levels in scale space

for i in range(num_level):
    current_sigma = current_sigma * scale
    scale_space[:, :, i] = np.square(current_sigma * current_sigma * (gaussian_laplace(read_image, sigma = current_sigma, mode = 'mirror')))

end_time = time.time()
print("time: ", end_time - start_time)

In [ ]:
scale_space_after_rank_filter = np.empty((image_height, image_width, num_level))
for i in range(num_level):
    scale_space_after_rank_filter[:, :, i] = rank_filter(scale_space[:, :, i], -1, size = 3, mode='mirror')
    # zero out changed elements in the 2d layer
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter[i, j, k] != scale_space[i, j, k] :
                scale_space_after_rank_filter[i, j, k]  = 0.0

In [ ]:
scale_space_after_rank_filter_3d = rank_filter(scale_space_after_rank_filter, -1, size = (compare_size, compare_size, compare_height), mode='mirror')
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter_3d[i, j, k] != scale_space_after_rank_filter[i, j, k]:
                scale_space_after_rank_filter_3d[i, j, k] = 0.0

In [ ]:
cx = []
cy = []
rad = []
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter_3d[i, j, k] != 0.0:
                cy.append(i)
                cx.append(j)
                rad.append(np.sqrt(2) * (scale * initial_sigma) * (scale ** k))

show_all_circles(read_image, cx = cx, cy = cy, rad = rad, color='r')

In [ ]:
# Downsample the image
sigma = 4
initial_scale = 2
current_scale = 2
down_scale = 1.1
num_level_2 = 12

compare_size = 25
compare_height = 3

In [ ]:
start_time = time.time()
scale_space_2 = np.empty((image_height, image_width, num_level_2))

for i in range(num_level_2):
    current_scale = current_scale * down_scale
    downscale_image = resize(read_image, (int((float(image_height) / current_scale)), int((float(image_width) / current_scale))), anti_aliasing=True)
    downscale_image_response = np.square(gaussian_laplace(downscale_image, sigma = sigma, mode = 'mirror'))
    downscale_image_response_upscale = resize(downscale_image_response, (image_height, image_width),anti_aliasing=True)
    scale_space_2[:, :, i] = downscale_image_response_upscale # store a matrix at level i
    
end_time = time.time()
print("time: ", end_time - start_time)

In [ ]:
scale_space_after_rank_filter_2 = np.empty((image_height, image_width, num_level_2))
for i in range(num_level_2):
    scale_space_after_rank_filter_2[:, :, i] = rank_filter(scale_space_2[:, :, i], -1, size = 3, mode='mirror')
    # zero out changed elements in the 2d layer
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_2[i, j, k] != scale_space_2[i, j, k] :
                scale_space_after_rank_filter_2[i, j, k]  = 0.0

In [ ]:
scale_space_after_rank_filter_3d_2 = rank_filter(scale_space_after_rank_filter_2, -1, size = (compare_size, compare_size, compare_height), mode='mirror')
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_3d_2[i, j, k] != scale_space_after_rank_filter_2[i, j, k]:
                scale_space_after_rank_filter_3d_2[i, j, k] = 0.0

In [ ]:
cx_2 = []
cy_2 = []
rad_2 = []
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_3d_2[i, j, k] != 0.0:
                cy_2.append(i)
                cx_2.append(j)
                rad_2.append(np.sqrt(2) * sigma * (((down_scale ** 2) ** k)))

show_all_circles(read_image, cx = cx_2, cy = cy_2, rad = rad_2, color='r')

# Einstein

In [ ]:
read_image = cv2.imread('assignment2_images/einstein.jpg', 0)
# normalize to 0 to 1
read_image = read_image / 255
plt.imshow(read_image, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

In [ ]:
image_height = (read_image.shape)[0]
image_width = (read_image.shape)[1]

In [ ]:
# Change filter size
initial_sigma = 0.9
current_sigma = 0.9
scale = 1.3
num_level = 15
compare_size = 35
compare_height = 3

In [ ]:
start_time = time.time()
scale_space = np.empty((image_height, image_width, num_level)) # [h,w] - dimensions of image, n - number of levels in scale space

for i in range(num_level):
    current_sigma = current_sigma * scale
    scale_space[:, :, i] = np.square(current_sigma * current_sigma * (gaussian_laplace(read_image, sigma = current_sigma, mode = 'mirror')))

end_time = time.time()
print("time: ", end_time - start_time)

In [ ]:
scale_space_after_rank_filter = np.empty((image_height, image_width, num_level))
for i in range(num_level):
    scale_space_after_rank_filter[:, :, i] = rank_filter(scale_space[:, :, i], -1, size = 3, mode='mirror')
    # zero out changed elements in the 2d layer
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter[i, j, k] != scale_space[i, j, k] :
                scale_space_after_rank_filter[i, j, k]  = 0.0

In [ ]:
scale_space_after_rank_filter_3d = rank_filter(scale_space_after_rank_filter, -1, size = (compare_size, compare_size, compare_height), mode='mirror')
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter_3d[i, j, k] != scale_space_after_rank_filter[i, j, k]:
                scale_space_after_rank_filter_3d[i, j, k] = 0.0

In [ ]:
cx = []
cy = []
rad = []
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter_3d[i, j, k] != 0.0:
                cy.append(i)
                cx.append(j)
                rad.append(np.sqrt(2) * (scale * initial_sigma) * (scale ** k))

show_all_circles(read_image, cx = cx, cy = cy, rad = rad, color='r')

In [ ]:
# Downsample the image
sigma = 4
initial_scale = 2
current_scale = 2
down_scale = 1.1
num_level_2 = 15

compare_size = 35
compare_height = 3

In [ ]:
start_time = time.time()
scale_space_2 = np.empty((image_height, image_width, num_level_2))

for i in range(num_level_2):
    current_scale = current_scale * down_scale
    downscale_image = resize(read_image, (int((float(image_height) / current_scale)), int((float(image_width) / current_scale))), anti_aliasing=True)
    downscale_image_response = np.square(gaussian_laplace(downscale_image, sigma = sigma, mode = 'mirror'))
    downscale_image_response_upscale = resize(downscale_image_response, (image_height, image_width),anti_aliasing=True)
    scale_space_2[:, :, i] = downscale_image_response_upscale # store a matrix at level i
    
end_time = time.time()
print("time: ", end_time - start_time)

In [ ]:
scale_space_after_rank_filter_2 = np.empty((image_height, image_width, num_level_2))
for i in range(num_level_2):
    scale_space_after_rank_filter_2[:, :, i] = rank_filter(scale_space_2[:, :, i], -1, size = 3, mode='mirror')
    # zero out changed elements in the 2d layer
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_2[i, j, k] != scale_space_2[i, j, k] :
                scale_space_after_rank_filter_2[i, j, k]  = 0.0

In [ ]:
scale_space_after_rank_filter_3d_2 = rank_filter(scale_space_after_rank_filter_2, -1, size = (compare_size, compare_size, compare_height), mode='mirror')
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_3d_2[i, j, k] != scale_space_after_rank_filter_2[i, j, k]:
                scale_space_after_rank_filter_3d_2[i, j, k] = 0.0

In [ ]:
cx_2 = []
cy_2 = []
rad_2 = []
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_3d_2[i, j, k] != 0.0:
                cy_2.append(i)
                cx_2.append(j)
                rad_2.append(np.sqrt(2) * sigma * (((down_scale ** 2) ** k)))

show_all_circles(read_image, cx = cx_2, cy = cy_2, rad = rad_2, color='r')

# Fishes

In [ ]:
read_image = cv2.imread('assignment2_images/fishes.jpg', 0)
# normalize to 0 to 1
read_image = read_image / 255
plt.imshow(read_image, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

In [ ]:
image_height = (read_image.shape)[0]
image_width = (read_image.shape)[1]

In [ ]:
# Change filter size
initial_sigma = 0.9
current_sigma = 0.9
scale = 1.3
num_level = 15
compare_size = 30
compare_height = 3

In [ ]:
start_time = time.time()
scale_space = np.empty((image_height, image_width, num_level)) # [h,w] - dimensions of image, n - number of levels in scale space

for i in range(num_level):
    current_sigma = current_sigma * scale
    scale_space[:, :, i] = np.square(current_sigma * current_sigma * (gaussian_laplace(read_image, sigma = current_sigma, mode = 'mirror')))

end_time = time.time()
print("time: ", end_time - start_time)

In [ ]:
scale_space_after_rank_filter = np.empty((image_height, image_width, num_level))
for i in range(num_level):
    scale_space_after_rank_filter[:, :, i] = rank_filter(scale_space[:, :, i], -1, size = 3, mode='mirror')
    # zero out changed elements in the 2d layer
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter[i, j, k] != scale_space[i, j, k] :
                scale_space_after_rank_filter[i, j, k]  = 0.0

In [ ]:
scale_space_after_rank_filter_3d = rank_filter(scale_space_after_rank_filter, -1, size = (compare_size, compare_size, compare_height), mode='mirror')
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter_3d[i, j, k] != scale_space_after_rank_filter[i, j, k]:
                scale_space_after_rank_filter_3d[i, j, k] = 0.0

In [ ]:
cx = []
cy = []
rad = []
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter_3d[i, j, k] != 0.0:
                cy.append(i)
                cx.append(j)
                rad.append(np.sqrt(2) * (scale * initial_sigma) * (scale ** k))

show_all_circles(read_image, cx = cx, cy = cy, rad = rad, color='r')

In [ ]:
# Downsample the image
sigma = 3.5
initial_scale = 2
current_scale = 2
down_scale = 1.1
num_level_2 = 15

compare_size = 30
compare_height = 3

In [ ]:
start_time = time.time()
scale_space_2 = np.empty((image_height, image_width, num_level_2))

for i in range(num_level_2):
    current_scale = current_scale * down_scale
    downscale_image = resize(read_image, (int((float(image_height) / current_scale)), int((float(image_width) / current_scale))), anti_aliasing=True)
    downscale_image_response = np.square(gaussian_laplace(downscale_image, sigma = sigma, mode = 'mirror'))
    downscale_image_response_upscale = resize(downscale_image_response, (image_height, image_width),anti_aliasing=True)
    scale_space_2[:, :, i] = downscale_image_response_upscale # store a matrix at level i
    
end_time = time.time()
print("time: ", end_time - start_time)

In [ ]:
scale_space_after_rank_filter_2 = np.empty((image_height, image_width, num_level_2))
for i in range(num_level_2):
    scale_space_after_rank_filter_2[:, :, i] = rank_filter(scale_space_2[:, :, i], -1, size = 3, mode='mirror')
    # zero out changed elements in the 2d layer
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_2[i, j, k] != scale_space_2[i, j, k] :
                scale_space_after_rank_filter_2[i, j, k]  = 0.0

In [ ]:
scale_space_after_rank_filter_3d_2 = rank_filter(scale_space_after_rank_filter_2, -1, size = (compare_size, compare_size, compare_height), mode='mirror')
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_3d_2[i, j, k] != scale_space_after_rank_filter_2[i, j, k]:
                scale_space_after_rank_filter_3d_2[i, j, k] = 0.0

In [ ]:
cx_2 = []
cy_2 = []
rad_2 = []
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_3d_2[i, j, k] != 0.0:
                cy_2.append(i)
                cx_2.append(j)
                rad_2.append(np.sqrt(2) * sigma * (((down_scale ** 2) ** k)))

show_all_circles(read_image, cx = cx_2, cy = cy_2, rad = rad_2, color='r')

# Sunflowers

In [ ]:
read_image = cv2.imread('assignment2_images/sunflowers.jpg', 0)
# normalize to 0 to 1
read_image = read_image / 255
plt.imshow(read_image, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

In [ ]:
image_height = (read_image.shape)[0]
image_width = (read_image.shape)[1]

In [ ]:
# Change filter size
initial_sigma = 0.9
current_sigma = 0.9
scale = 1.3
num_level = 15
compare_size = 35
compare_height = 3

In [ ]:
start_time = time.time()
scale_space = np.empty((image_height, image_width, num_level)) # [h,w] - dimensions of image, n - number of levels in scale space

for i in range(num_level):
    current_sigma = current_sigma * scale
    scale_space[:, :, i] = np.square(current_sigma * current_sigma * (gaussian_laplace(read_image, sigma = current_sigma, mode = 'mirror')))

end_time = time.time()
print("time: ", end_time - start_time)

In [ ]:
scale_space_after_rank_filter = np.empty((image_height, image_width, num_level))
for i in range(num_level):
    scale_space_after_rank_filter[:, :, i] = rank_filter(scale_space[:, :, i], -1, size = 3, mode='mirror')
    # zero out changed elements in the 2d layer
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter[i, j, k] != scale_space[i, j, k] :
                scale_space_after_rank_filter[i, j, k]  = 0.0

In [ ]:
scale_space_after_rank_filter_3d = rank_filter(scale_space_after_rank_filter, -1, size = (compare_size, compare_size, compare_height), mode='mirror')
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter_3d[i, j, k] != scale_space_after_rank_filter[i, j, k]:
                scale_space_after_rank_filter_3d[i, j, k] = 0.0

In [ ]:
cx = []
cy = []
rad = []
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter_3d[i, j, k] != 0.0:
                cy.append(i)
                cx.append(j)
                rad.append(np.sqrt(2) * (scale * initial_sigma) * (scale ** k))

show_all_circles(read_image, cx = cx, cy = cy, rad = rad, color='r')

In [ ]:
# Downsample the image
sigma = 2.8
initial_scale = 2
current_scale = 2
down_scale = 1.1
num_level_2 = 15

compare_size = 35
compare_height = 3

In [ ]:
start_time = time.time()
scale_space_2 = np.empty((image_height, image_width, num_level_2))

for i in range(num_level_2):
    current_scale = current_scale * down_scale
    downscale_image = resize(read_image, (int((float(image_height) / current_scale)), int((float(image_width) / current_scale))), anti_aliasing=True)
    downscale_image_response = np.square(gaussian_laplace(downscale_image, sigma = sigma, mode = 'mirror'))
    downscale_image_response_upscale = resize(downscale_image_response, (image_height, image_width),anti_aliasing=True)
    scale_space_2[:, :, i] = downscale_image_response_upscale # store a matrix at level i
    
end_time = time.time()
print("time: ", end_time - start_time)

In [ ]:
scale_space_after_rank_filter_2 = np.empty((image_height, image_width, num_level_2))
for i in range(num_level_2):
    scale_space_after_rank_filter_2[:, :, i] = rank_filter(scale_space_2[:, :, i], -1, size = 3, mode='mirror')
    # zero out changed elements in the 2d layer
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_2[i, j, k] != scale_space_2[i, j, k] :
                scale_space_after_rank_filter_2[i, j, k]  = 0.0

In [ ]:
scale_space_after_rank_filter_3d_2 = rank_filter(scale_space_after_rank_filter_2, -1, size = (compare_size, compare_size, compare_height), mode='mirror')
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_3d_2[i, j, k] != scale_space_after_rank_filter_2[i, j, k]:
                scale_space_after_rank_filter_3d_2[i, j, k] = 0.0

In [ ]:
cx_2 = []
cy_2 = []
rad_2 = []
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_3d_2[i, j, k] != 0.0:
                cy_2.append(i)
                cx_2.append(j)
                rad_2.append(np.sqrt(2) * sigma * (((down_scale ** 2) ** k)))

show_all_circles(read_image, cx = cx_2, cy = cy_2, rad = rad_2, color='r')

# My choice 1 - Apples

In [ ]:
read_image = cv2.imread('assignment2_images/apples.jpg', 0)
# normalize to 0 to 1
read_image = read_image / 255
plt.imshow(read_image, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

In [ ]:
image_height = (read_image.shape)[0]
image_width = (read_image.shape)[1]

In [ ]:
# Change filter size
initial_sigma = 1.2
current_sigma = 1.2
scale = 1.2
num_level = 12
compare_size = 30
compare_height = 3

In [ ]:
start_time = time.time()
scale_space = np.empty((image_height, image_width, num_level)) # [h,w] - dimensions of image, n - number of levels in scale space

for i in range(num_level):
    current_sigma = current_sigma * scale
    scale_space[:, :, i] = np.square(current_sigma * current_sigma * (gaussian_laplace(read_image, sigma = current_sigma, mode = 'mirror')))

end_time = time.time()
print("time: ", end_time - start_time)

In [ ]:
scale_space_after_rank_filter = np.empty((image_height, image_width, num_level))
for i in range(num_level):
    scale_space_after_rank_filter[:, :, i] = rank_filter(scale_space[:, :, i], -1, size = 3, mode='mirror')
    # zero out changed elements in the 2d layer
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter[i, j, k] != scale_space[i, j, k] :
                scale_space_after_rank_filter[i, j, k]  = 0.0

In [ ]:
scale_space_after_rank_filter_3d = rank_filter(scale_space_after_rank_filter, -1, size = (compare_size, compare_size, compare_height), mode='mirror')
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter_3d[i, j, k] != scale_space_after_rank_filter[i, j, k]:
                scale_space_after_rank_filter_3d[i, j, k] = 0.0

In [ ]:
cx = []
cy = []
rad = []
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter_3d[i, j, k] != 0.0:
                cy.append(i)
                cx.append(j)
                rad.append(np.sqrt(2) * (scale * initial_sigma) * (scale ** k))

show_all_circles(read_image, cx = cx, cy = cy, rad = rad, color='r')

In [ ]:
# Downsample the image
sigma = 1.2
initial_scale = 1.2
current_scale = 1.2
down_scale = 1.1
num_level_2 = 12

compare_size = 40
compare_height = 3

In [ ]:
start_time = time.time()
scale_space_2 = np.empty((image_height, image_width, num_level_2))

for i in range(num_level_2):
    current_scale = current_scale * down_scale
    downscale_image = resize(read_image, (int((float(image_height) / current_scale)), int((float(image_width) / current_scale))), anti_aliasing=True)
    downscale_image_response = np.square(gaussian_laplace(downscale_image, sigma = sigma, mode = 'mirror'))
    downscale_image_response_upscale = resize(downscale_image_response, (image_height, image_width),anti_aliasing=True)
    scale_space_2[:, :, i] = downscale_image_response_upscale # store a matrix at level i
    
end_time = time.time()
print("time: ", end_time - start_time)

In [ ]:
scale_space_after_rank_filter_2 = np.empty((image_height, image_width, num_level_2))
for i in range(num_level_2):
    scale_space_after_rank_filter_2[:, :, i] = rank_filter(scale_space_2[:, :, i], -1, size = 3, mode='mirror')
    # zero out changed elements in the 2d layer
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_2[i, j, k] != scale_space_2[i, j, k] :
                scale_space_after_rank_filter_2[i, j, k]  = 0.0

In [ ]:
scale_space_after_rank_filter_3d_2 = rank_filter(scale_space_after_rank_filter_2, -1, size = (compare_size, compare_size, compare_height), mode='mirror')
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_3d_2[i, j, k] != scale_space_after_rank_filter_2[i, j, k]:
                scale_space_after_rank_filter_3d_2[i, j, k] = 0.0

In [ ]:
cx_2 = []
cy_2 = []
rad_2 = []
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_3d_2[i, j, k] != 0.0:
                cy_2.append(i)
                cx_2.append(j)
                rad_2.append(np.sqrt(2) * sigma * (((down_scale ** 2) ** k)))

show_all_circles(read_image, cx = cx_2, cy = cy_2, rad = rad_2, color='r')

# My choice 2 - Octupus

In [ ]:
read_image = cv2.imread('assignment2_images/octupus.jpg', 0)
# normalize to 0 to 1
read_image = read_image / 255
plt.imshow(read_image, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

In [ ]:
image_height = (read_image.shape)[0]
image_width = (read_image.shape)[1]

In [ ]:
# Change filter size
initial_sigma = 1.2
current_sigma = 1.2
scale = 1.2
num_level = 12
compare_size = 30
compare_height = 3

In [ ]:
start_time = time.time()
scale_space = np.empty((image_height, image_width, num_level)) # [h,w] - dimensions of image, n - number of levels in scale space

for i in range(num_level):
    current_sigma = current_sigma * scale
    scale_space[:, :, i] = np.square(current_sigma * current_sigma * (gaussian_laplace(read_image, sigma = current_sigma, mode = 'mirror')))

end_time = time.time()
print("time: ", end_time - start_time)

In [ ]:
scale_space_after_rank_filter = np.empty((image_height, image_width, num_level))
for i in range(num_level):
    scale_space_after_rank_filter[:, :, i] = rank_filter(scale_space[:, :, i], -1, size = 3, mode='mirror')
    # zero out changed elements in the 2d layer
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter[i, j, k] != scale_space[i, j, k] :
                scale_space_after_rank_filter[i, j, k]  = 0.0

In [ ]:
scale_space_after_rank_filter_3d = rank_filter(scale_space_after_rank_filter, -1, size = (compare_size, compare_size, compare_height), mode='mirror')
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter_3d[i, j, k] != scale_space_after_rank_filter[i, j, k]:
                scale_space_after_rank_filter_3d[i, j, k] = 0.0

In [ ]:
cx = []
cy = []
rad = []
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter_3d[i, j, k] != 0.0:
                cy.append(i)
                cx.append(j)
                rad.append(np.sqrt(2) * (scale * initial_sigma) * (scale ** k))

show_all_circles(read_image, cx = cx, cy = cy, rad = rad, color='r')

In [ ]:
# Downsample the image
sigma = 2.2
initial_scale = 1.1
current_scale = 1.1
down_scale = 1.1
num_level_2 = 12

compare_size = 40
compare_height = 3

In [ ]:
start_time = time.time()
scale_space_2 = np.empty((image_height, image_width, num_level_2))

for i in range(num_level_2):
    current_scale = current_scale * down_scale
    downscale_image = resize(read_image, (int((float(image_height) / current_scale)), int((float(image_width) / current_scale))), anti_aliasing=True)
    downscale_image_response = np.square(gaussian_laplace(downscale_image, sigma = sigma, mode = 'mirror'))
    downscale_image_response_upscale = resize(downscale_image_response, (image_height, image_width),anti_aliasing=True)
    scale_space_2[:, :, i] = downscale_image_response_upscale # store a matrix at level i
    
end_time = time.time()
print("time: ", end_time - start_time)

In [ ]:
scale_space_after_rank_filter_2 = np.empty((image_height, image_width, num_level_2))
for i in range(num_level_2):
    scale_space_after_rank_filter_2[:, :, i] = rank_filter(scale_space_2[:, :, i], -1, size = 3, mode='mirror')
    # zero out changed elements in the 2d layer
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_2[i, j, k] != scale_space_2[i, j, k] :
                scale_space_after_rank_filter_2[i, j, k]  = 0.0

In [ ]:
scale_space_after_rank_filter_3d_2 = rank_filter(scale_space_after_rank_filter_2, -1, size = (compare_size, compare_size, compare_height), mode='mirror')
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_3d_2[i, j, k] != scale_space_after_rank_filter_2[i, j, k]:
                scale_space_after_rank_filter_3d_2[i, j, k] = 0.0

In [ ]:
cx_2 = []
cy_2 = []
rad_2 = []
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_3d_2[i, j, k] != 0.0:
                cy_2.append(i)
                cx_2.append(j)
                rad_2.append(np.sqrt(2) * sigma * (((down_scale ** 2) ** k)))

show_all_circles(read_image, cx = cx_2, cy = cy_2, rad = rad_2, color='r')

# My choice 3 - Go Board Game

In [ ]:
read_image = cv2.imread('assignment2_images/go_game.png', 0)
# normalize to 0 to 1
read_image = read_image / 255
plt.imshow(read_image, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

In [ ]:
image_height = (read_image.shape)[0]
image_width = (read_image.shape)[1]

In [ ]:
# Change filter size
initial_sigma = 2.1
current_sigma = 2.1
scale = 1.2
num_level = 10
compare_size = 45
compare_height = 3

In [ ]:
start_time = time.time()
scale_space = np.empty((image_height, image_width, num_level)) # [h,w] - dimensions of image, n - number of levels in scale space

for i in range(num_level):
    current_sigma = current_sigma * scale
    scale_space[:, :, i] = np.square(current_sigma * current_sigma * (gaussian_laplace(read_image, sigma = current_sigma, mode = 'mirror')))

end_time = time.time()
print("time: ", end_time - start_time)

In [ ]:
scale_space_after_rank_filter = np.empty((image_height, image_width, num_level))
for i in range(num_level):
    scale_space_after_rank_filter[:, :, i] = rank_filter(scale_space[:, :, i], -1, size = 3, mode='mirror')
    # zero out changed elements in the 2d layer
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter[i, j, k] != scale_space[i, j, k] :
                scale_space_after_rank_filter[i, j, k]  = 0.0

In [ ]:
scale_space_after_rank_filter_3d = rank_filter(scale_space_after_rank_filter, -1, size = (compare_size, compare_size, compare_height), mode='mirror')
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter_3d[i, j, k] != scale_space_after_rank_filter[i, j, k]:
                scale_space_after_rank_filter_3d[i, j, k] = 0.0

In [ ]:
cx = []
cy = []
rad = []
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter_3d[i, j, k] != 0.0:
                cy.append(i)
                cx.append(j)
                rad.append(np.sqrt(2) * (scale * initial_sigma) * (scale ** k))

show_all_circles(read_image, cx = cx, cy = cy, rad = rad, color='r')

In [ ]:
# Downsample the image
sigma = 2.1
initial_scale = 1.1
current_scale = 1.1
down_scale = 1.1
num_level_2 = 10

compare_size = 45
compare_height = 5

In [ ]:
start_time = time.time()
scale_space_2 = np.empty((image_height, image_width, num_level_2))

for i in range(num_level_2):
    current_scale = current_scale * down_scale
    downscale_image = resize(read_image, (int((float(image_height) / current_scale)), int((float(image_width) / current_scale))), anti_aliasing=True)
    downscale_image_response = np.square(gaussian_laplace(downscale_image, sigma = sigma, mode = 'mirror'))
    downscale_image_response_upscale = resize(downscale_image_response, (image_height, image_width),anti_aliasing=True)
    scale_space_2[:, :, i] = downscale_image_response_upscale # store a matrix at level i
    
end_time = time.time()
print("time: ", end_time - start_time)

In [ ]:
scale_space_after_rank_filter_2 = np.empty((image_height, image_width, num_level_2))
for i in range(num_level_2):
    scale_space_after_rank_filter_2[:, :, i] = rank_filter(scale_space_2[:, :, i], -1, size = 3, mode='mirror')
    # zero out changed elements in the 2d layer
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_2[i, j, k] != scale_space_2[i, j, k] :
                scale_space_after_rank_filter_2[i, j, k]  = 0.0

In [ ]:
scale_space_after_rank_filter_3d_2 = rank_filter(scale_space_after_rank_filter_2, -1, size = (compare_size, compare_size, compare_height), mode='mirror')
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_3d_2[i, j, k] != scale_space_after_rank_filter_2[i, j, k]:
                scale_space_after_rank_filter_3d_2[i, j, k] = 0.0

In [ ]:
cx_2 = []
cy_2 = []
rad_2 = []
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_3d_2[i, j, k] != 0.0:
                cy_2.append(i)
                cx_2.append(j)
                rad_2.append(np.sqrt(2) * sigma * (((down_scale ** 2) ** k)))

show_all_circles(read_image, cx = cx_2, cy = cy_2, rad = rad_2, color='r')

# My choice 4 - Eggs

In [ ]:
read_image = cv2.imread('assignment2_images/eggs.jpg', 0)
# normalize to 0 to 1
read_image = read_image / 255
plt.imshow(read_image, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

In [ ]:
image_height = (read_image.shape)[0]
image_width = (read_image.shape)[1]

In [ ]:
# Change filter size
sigma = 2.1
initial_scale = 1.1
current_scale = 1.1
down_scale = 1.1
num_level_2 = 10

compare_size = 45
compare_height = 5

In [ ]:
start_time = time.time()
scale_space = np.empty((image_height, image_width, num_level)) # [h,w] - dimensions of image, n - number of levels in scale space

for i in range(num_level):
    current_sigma = current_sigma * scale
    scale_space[:, :, i] = np.square(current_sigma * current_sigma * (gaussian_laplace(read_image, sigma = current_sigma, mode = 'mirror')))

end_time = time.time()
print("time: ", end_time - start_time)

In [ ]:
scale_space_after_rank_filter = np.empty((image_height, image_width, num_level))
for i in range(num_level):
    scale_space_after_rank_filter[:, :, i] = rank_filter(scale_space[:, :, i], -1, size = 3, mode='mirror')
    # zero out changed elements in the 2d layer
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter[i, j, k] != scale_space[i, j, k] :
                scale_space_after_rank_filter[i, j, k]  = 0.0

In [ ]:
scale_space_after_rank_filter_3d = rank_filter(scale_space_after_rank_filter, -1, size = (compare_size, compare_size, compare_height), mode='mirror')
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter_3d[i, j, k] != scale_space_after_rank_filter[i, j, k]:
                scale_space_after_rank_filter_3d[i, j, k] = 0.0

In [ ]:
cx = []
cy = []
rad = []
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level):
            if scale_space_after_rank_filter_3d[i, j, k] != 0.0:
                cy.append(i)
                cx.append(j)
                rad.append(np.sqrt(2) * (scale * initial_sigma) * (scale ** k))

show_all_circles(read_image, cx = cx, cy = cy, rad = rad, color='r')

In [ ]:
# Downsample the image
sigma = 3.5
initial_scale = 1.1
current_scale = 1.1
down_scale = 1.1
num_level_2 = 10

compare_size = 45
compare_height = 5

In [ ]:
start_time = time.time()
scale_space_2 = np.empty((image_height, image_width, num_level_2))

for i in range(num_level_2):
    current_scale = current_scale * down_scale
    downscale_image = resize(read_image, (int((float(image_height) / current_scale)), int((float(image_width) / current_scale))), anti_aliasing=True)
    downscale_image_response = np.square(gaussian_laplace(downscale_image, sigma = sigma, mode = 'mirror'))
    downscale_image_response_upscale = resize(downscale_image_response, (image_height, image_width),anti_aliasing=True)
    scale_space_2[:, :, i] = downscale_image_response_upscale # store a matrix at level i
    
end_time = time.time()
print("time: ", end_time - start_time)

In [ ]:
scale_space_after_rank_filter_2 = np.empty((image_height, image_width, num_level_2))
for i in range(num_level_2):
    scale_space_after_rank_filter_2[:, :, i] = rank_filter(scale_space_2[:, :, i], -1, size = 3, mode='mirror')
    # zero out changed elements in the 2d layer
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_2[i, j, k] != scale_space_2[i, j, k] :
                scale_space_after_rank_filter_2[i, j, k]  = 0.0

In [ ]:
scale_space_after_rank_filter_3d_2 = rank_filter(scale_space_after_rank_filter_2, -1, size = (compare_size, compare_size, compare_height), mode='mirror')
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_3d_2[i, j, k] != scale_space_after_rank_filter_2[i, j, k]:
                scale_space_after_rank_filter_3d_2[i, j, k] = 0.0

In [ ]:
cx_2 = []
cy_2 = []
rad_2 = []
for i in range(image_height):
    for j in range(image_width):
        for k in range(num_level_2):
            if scale_space_after_rank_filter_3d_2[i, j, k] != 0.0:
                cy_2.append(i)
                cx_2.append(j)
                rad_2.append(np.sqrt(2) * sigma * (((down_scale ** 2) ** k)))

show_all_circles(read_image, cx = cx_2, cy = cy_2, rad = rad_2, color='r')